In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

C:\Temp\ipykernel_19384\1028975186.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
root = '/CSRNet/ShanghaiTech/'

In [5]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root, 'part_A', 'train_data', 'images').replace('\\', '/')
part_A_test = os.path.join(root, 'part_A', 'test_data', 'images').replace('\\', '/')
part_B_train = os.path.join(root, 'part_B', 'train_data', 'images').replace('\\', '/')
part_B_test = os.path.join(root, 'part_B', 'test_data', 'images').replace('\\', '/')

path_sets = [part_A_train,part_A_test]
print(path_sets)

['/CSRNet/ShanghaiTech/part_A/train_data/images', '/CSRNet/ShanghaiTech/part_A/test_data/images']


In [6]:
img_paths = []

for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_path = img_path.replace('\\', '/')
        img_paths.append(img_path)

print(img_paths)

['/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_1.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_10.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_100.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_101.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_102.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_103.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_104.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_105.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_106.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_107.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_108.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_109.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_11.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_110.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_111.jpg', '/CSRNet/ShanghaiTech/part_A/train_data/images/IMG_112.jpg', '/CSRNet/ShanghaiTech/part_

In [7]:
model = CSRNet()

C:\Users\jun52\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jun52\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
model = model.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [9]:
checkpoint = torch.load('Amodel_best.pth.tar')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [39]:
model.load_state_dict(checkpoint['state_dict'])

In [45]:
mae = 0
for i in range(len(img_paths)):
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))

    img[0,:,:]=img[0,:,:]-92.8207477031
    img[1,:,:]=img[1,:,:]-95.2757037428
    img[2,:,:]=img[2,:,:]-104.877445883
    img = img.cuda()
    #img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print i,mae
print mae/len(img_paths)

0 15.50390625
1 60.9075317383
2 220.511169434
3 239.312469482
4 252.252349854
5 272.965286255
6 457.101577759
7 651.92250061
8 681.363113403
9 785.472061157
10 838.996322632
11 1012.11277771
12 1073.18791199
13 1074.72886658
14 1139.53701782
15 1201.55630493
16 1316.97366333
17 1447.83328247
18 1578.5967865
19 1622.11299896
20 1650.6510849
21 1756.23152924
22 1810.32888031
23 1816.16628265
24 1864.3579483
25 1886.67713928
26 1942.00212097
27 1995.17939758
28 2031.85020447
29 2236.5879364
30 2265.58026123
31 2272.81892395
32 2333.38215637
33 2489.37367249
34 2560.18891907
35 2580.02906799
36 2588.45735168
37 2661.36177063
38 2788.47312927
39 2900.07542419
40 2900.83190918
41 2976.19485474
42 2995.64665985
43 3058.02416229
44 3103.91133881
45 3241.52921295
46 3906.06101227
47 3918.4709549
48 3948.23314667
49 3953.28383636
50 3989.9439621
51 4093.59087372
52 4180.21788788
53 4289.41963959
54 4291.79859161
55 4302.33109283
56 4339.80475616
57 4543.3482132
58 4626.09952545
59 4684.85929108
